# Project Name ---> HeartGuard Predictor
## Classification  Basic model 

In [154]:
# importing requred libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score , roc_curve , f1_score , class_likelihood_ratios
from sklearn.metrics import confusion_matrix

In [155]:
# loading datsets CSV

file_path = r"C:\Users\Ravichandran\OneDrive\Desktop\heart_disease_uci.csv"
heart_data = pd.read_csv(file_path)


In [156]:
heart_data

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [157]:
heart_data.columns

Index(['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs',
       'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')

In [158]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [159]:
heart_data.isnull().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [160]:
heart_data.drop(['id','dataset'] , axis=1, inplace=True)

In [161]:
heart_data.dropna(subset=['trestbps', 'chol', 'fbs','restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal',] , axis=0 , inplace=True)

In [162]:
heart_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,68,Male,asymptomatic,144.0,193.0,True,normal,141.0,False,3.4,flat,2.0,reversable defect,2
300,57,Male,asymptomatic,130.0,131.0,False,normal,115.0,True,1.2,flat,1.0,reversable defect,3
301,57,Female,atypical angina,130.0,236.0,False,lv hypertrophy,174.0,False,0.0,flat,1.0,normal,1
508,47,Male,asymptomatic,150.0,226.0,False,normal,98.0,True,1.5,flat,0.0,reversable defect,1


In [163]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [164]:
# Checking columns datatypes 
heart_data.dtypes

age           int64
sex          object
cp           object
trestbps    float64
chol        float64
fbs          object
restecg      object
thalch      float64
exang        object
oldpeak     float64
slope        object
ca          float64
thal         object
num           int64
dtype: object

## Encoding all categorical columns

In [165]:
future_category = ['sex' ,'cp','fbs','restecg','exang','slope','thal']
one_hot_encoder = OneHotEncoder(sparse_output=False)



In [166]:
# adding column Transformers for encoded colunms

ct = ColumnTransformer(transformers=[
    ('cat' , one_hot_encoder, future_category )]) # we have to maintain order Name, Transformer, Colum

ct.set_output(transform='pandas')
encoded_heart_data = ct.fit_transform(heart_data)
encoded_heart_data.head()

,cat__sex_Female,cat__sex_Male,cat__cp_asymptomatic,cat__cp_atypical angina,cat__cp_non-anginal,cat__cp_typical angina,cat__fbs_False,cat__fbs_True,cat__restecg_lv hypertrophy,cat__restecg_normal,cat__restecg_st-t abnormality,cat__exang_False,cat__exang_True,cat__slope_downsloping,cat__slope_flat,cat__slope_upsloping,cat__thal_fixed defect,cat__thal_normal,cat__thal_reversable defect
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [167]:
encoded_heart_data

,cat__sex_Female,cat__sex_Male,cat__cp_asymptomatic,cat__cp_atypical angina,cat__cp_non-anginal,cat__cp_typical angina,cat__fbs_False,cat__fbs_True,cat__restecg_lv hypertrophy,cat__restecg_normal,cat__restecg_st-t abnormality,cat__exang_False,cat__exang_True,cat__slope_downsloping,cat__slope_flat,cat__slope_upsloping,cat__thal_fixed defect,cat__thal_normal,cat__thal_reversable defect
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
300,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
301,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
508,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [168]:
encoded_heart_data.columns

Index(['cat__sex_Female', 'cat__sex_Male', 'cat__cp_asymptomatic',
       'cat__cp_atypical angina', 'cat__cp_non-anginal',
       'cat__cp_typical angina', 'cat__fbs_False', 'cat__fbs_True',
       'cat__restecg_lv hypertrophy', 'cat__restecg_normal',
       'cat__restecg_st-t abnormality', 'cat__exang_False', 'cat__exang_True',
       'cat__slope_downsloping', 'cat__slope_flat', 'cat__slope_upsloping',
       'cat__thal_fixed defect', 'cat__thal_normal',
       'cat__thal_reversable defect'],
      dtype='object')

In [169]:
encoded_heart_data.isnull().sum()

cat__sex_Female                  0
cat__sex_Male                    0
cat__cp_asymptomatic             0
cat__cp_atypical angina          0
cat__cp_non-anginal              0
cat__cp_typical angina           0
cat__fbs_False                   0
cat__fbs_True                    0
cat__restecg_lv hypertrophy      0
cat__restecg_normal              0
cat__restecg_st-t abnormality    0
cat__exang_False                 0
cat__exang_True                  0
cat__slope_downsloping           0
cat__slope_flat                  0
cat__slope_upsloping             0
cat__thal_fixed defect           0
cat__thal_normal                 0
cat__thal_reversable defect      0
dtype: int64